In [1]:
from argparse import ArgumentParser
from model.dgl_gcn import LigandDataset_dgl, collate_fn, MolecularGCN
from utils.package import plot_fit_confidence_bond
from pytorch_lightning import Trainer
import pytorch_lightning as pl
from collections import defaultdict
from dgl.dataloading import GraphDataLoader
from torch.utils.data import random_split
import torch
import numpy as np
import wandb
import time
from sklearn.metrics import r2_score
import matplotlib
import matplotlib.pyplot as plt
import os
matplotlib.use('Agg')

In [ ]:
project = '3dgnn'
wandb.login(key='local-8fe6e6b5840c4c05aaaf6aac5ca8c1fb58abbd1f', host='http://localhost:8080')
wandb.init(project=project, save_code=True, dir='log')

In [2]:
# %%wandb
def main(hparams):

    model_name = f'3dgnn-dim-{hparams.dim}-' + time.strftime("%Y%m%d_%H%M%S", time.localtime())
    dict_args = vars(hparams)
    model = MolecularGCN(**dict_args)
    # logger
    wandb_logger = pl.loggers.WandbLogger()
    # callbacks
    # early stopping
    early_stopping = pl.callbacks.early_stopping.EarlyStopping(monitor='val_loss', patience=20, mode='min')
    # checkpoint
    checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor='val_loss', mode='min', save_last=True,
                                                         dirpath='checkpoints', filename=model_name)
    if hparams.checkpoint == None:
        trainer = Trainer.from_argparse_args(hparams, logger=wandb_logger, auto_lr_find=True, callbacks=[early_stopping, checkpoint_callback])
    else:
        trainer = Trainer(resume_from_checkpoint=hparams.checkpoint, callbacks=[early_stopping])

    trainer.fit(model)
    # trainer.save_checkpoint(time.strftime("%Y%m%d_%H%M%S", time.localtime()) + ".ckpt")
    trainer.test(model, dataloaders=model.val_dataloader(), verbose=False)
    x = np.array(model.predictions['true'])
    y = np.array(model.predictions['pred'])
    val_r2 = r2_score(x, y)
    val_fig = plot_fit_confidence_bond(x, y, val_r2, annot=False)
    
    model.predictions = defaultdict(list)
    trainer.test(model, dataloaders=model.train_dataloader(), verbose=False)
    x = np.array(model.predictions['true'])
    y = np.array(model.predictions['pred'])
    train_r2 = r2_score(x, y)
    train_fig = plot_fit_confidence_bond(x, y, train_r2, annot=False)
    if False:
        wandb.log({'train_res': train_fig, 'val_res': val_fig})
        wandb.log({'val_r2': val_r2, 'train_r2':train_r2})
        wandb.finish()


if __name__ == "__main__":
    parser = ArgumentParser()
    parser.add_argument("--dim", type=int, default=256)
    parser.add_argument("--batch_size",type=int, default=256)
    parser.add_argument("--raw_dir",type=str, default=None)
    parser.add_argument("--save_dir",type=str, default=None)
    parser.add_argument("--dataset_name",type=str, default=None)
    parser.add_argument("--checkpoint",type=str, default=None)
    # parser.add_argument("--max_epoch",type=int, default=500)
    raw_dir = 'data'
    save_dir = 'data'
    # add model args
    dataset_name = 'in_man_exhaustiveness_96_dock_conformation'
    # dataset_name = 'data_test'
    parser = MolecularGCN.add_model_specific_args(parent_parser=parser)
    # add Trainer args
    parser = pl.Trainer.add_argparse_args(parser)
    args = parser.parse_args(['--dataset_name', dataset_name, '--raw_dir', raw_dir,'--save_dir', save_dir, '--learning_rate', '0.0001',
                                 '--gpus=1', '--max_epochs', '1000'])

    main(args)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: huabei. Use `wandb login --relogin` to force relogin


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/huabei/anaconda3/envs/smtr/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /mnt/e/Python_Project/SMTarRNA/project/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | embed_atom | Embedding  | 2.0 K 
1 | gamma      | ModuleList | 24    
2 | conv       | ModuleList | 795 K 
3 | dense      | ModuleList | 526 K 
4 | dense_out  | Linear     | 257   
------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.297     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/huabei/anaconda3/envs/smtr/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/huabei/anaconda3/envs/smtr/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/home/huabei/anaconda3/envs/smtr/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


TypeError: test() got an unexpected keyword argument 'dataloders'

In [3]:
print(__name__)

__main__
